In [42]:
import pandas as pd
import numpy as np
import sqlite3
import datetime
import sklearn
import sqlalchemy as sa
from sklearn.model_selection import TimeSeriesSplit

In [438]:
from collections import Counter
from sklearn.base import BaseEstimator
from tqdm import trange
from imblearn.combine import SMOTETomek
from sklearn.linear_model import LogisticRegression

In [3]:
trans_feat = pd.read_csv('../data/data1per.csv', index_col=0)

/home/alex/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/alex/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [359]:
class forget_model(BaseEstimator):
    def __init__(self, model=LogisticRegression(),n_splits=4, balance=None):
        self.models = [] # лист, в котором будут храниться обученные модели для каждого сплита
        self.n_splits = n_splits # число сплитов
        self.balance = balance # должен поддерживать большинство 
        # методов из imbalanced-learn, которые поддерживают fit_sample
        self.model = model # модели, поддерживающие fit, predict и predict_proba
    
    # inputs - уникальные значения дат
    def _iterate_megabatches(self, inputs, n_splits): # делает разбиение по времени без пересечений
        batchsize = len(inputs)//n_splits
        for start_idx in trange(0, len(inputs) - batchsize + 1, batchsize):
            excerpt = [start_idx, start_idx + batchsize-1]
            yield inputs[excerpt]
    
    # X - это весь датафрейм за два месяца ВМЕСТЕ с таргетом
    def fit(self, X):
        models = [] # пустой список моделей
        batch_df_i_neg = pd.DataFrame() # заготовки для не фрода в итом сплите
        batch_df_pos = pd.DataFrame() #  заготовка для фрода
        unique_TS_split = np.array(sorted(X.TS_indexer.unique())) # получаем число уникальных дат
        
        for train_index in self._iterate_megabatches(inputs=unique_TS_split, n_splits=self.n_splits): 
            batch_df_i_old_neg = batch_df_i_neg # сохраняем прошлуб базу не фрода
            batch_df_i = X[X.TS_indexer.isin(train_index)] # получаем базу данных итого сплита
            batch_df_i_pos = batch_df_i[batch_df_i.label==1] #  разбиваем ее на фрод
            batch_df_i_neg = batch_df_i[batch_df_i.label==0] #  и не фрод
            batch_df_pos = pd.concat([batch_df_pos, batch_df_i_pos]) # получаем данные не фрода для обучения
            batch_df_neg = pd.concat([batch_df_i_old_neg, batch_df_i_neg]) # получаем данные фрода для обучения
            df_i = pd.concat([batch_df_pos, batch_df_neg]) #  объединяем фрод и не фрод
            X_df_i = df_i.drop(['label', 'short_date','TS_indexer', 'user_id'], axis=1) #  выделяем обучение
            y_df_i = df_i.label #  и таргет
            if self.balance is None: #  без балансировки просто продолжаем
                pass
            else: 
                balancing = self.balance 
                X_df_balanced_i, y_df_balanced_i = balancing.fit_sample(X_df_i, y_df_i) # делаем перебалансировку
                X_df_i = X_df_balanced_i
                y_df_i = y_df_balanced_i

            model_i = self.model # итая модель
            model_i.fit(X_df_i, y_df_i) # обучаем итую модель

            
            models.append(model_i) #  сохраняем ее в лист
        self.models = models #  возвращаем в селф


    def predict(self, X_test): 
        pr = np.zeros(len(X_test)) # заготовка
        for model_i in self.models:
            pr_i = model_i.predict(X_test) # предсказания итой модели
            pr = np.vstack([pr,pr_i]) # объединяем предсказания
        predictions = np.array([Counter(x).most_common(1)[0][0] for x in pr[1:].T]) #  простое голосвание
        return predictions
    
    def predict_proba(self, X_test):
        pr = np.array([np.zeros(len(X_test))]*2).T #заготовка
        for model_i in self.models:
            pr_i = model_i.predict_proba(X_test) #  получаем вероятности итой модели
            pr = np.sum([pr, pr_i], axis = 0) 
        predictions = pr/len(self.models) #  получаем усредненные вероятности
        return predictions



In [154]:
trans_feat = pd.read_csv('../data/frst_chunk.csv', index_col=0)
trans_feat['TS_indexer'] = trans_feat.short_date
trans_feat.fillna(0, inplace=True)
trans_feat.drop(['event_id', 'event_time'], axis = 1, inplace=True)

/home/alex/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [372]:
m = forget_model(model=LogisticRegression(), n_splits=4, balance=SMOTETomek())

In [373]:
m.fit(trans_feat)

100%|██████████| 1/1 [00:00<00:00,  4.56it/s]


In [374]:
res = m.predict(trans_feat.drop(['label', 'short_date','TS_indexer', 'user_id'], axis=1))

In [376]:
sum(res)

2616.0

In [377]:
sum((res == trans_feat.label)[np.argwhere(trans_feat.label == 1).ravel()])

112

In [431]:
def train_val_ts_split(X, th=0.7, split_target=None): # X - датафрейм, в котором есть  TS_indexer
    unique_TS_split = np.array(sorted(X.TS_indexer.unique()))
    batchsize = int(len(unique_TS_split)*th)
    excerpt_train = [0, batchsize-1]
    excerpt_val = [batchsize, len(unique_TS_split)-1]
    d_train =  X[X.TS_indexer.isin(unique_TS_split[excerpt_train])]
    d_val = X[X.TS_indexer.isin(unique_TS_split[excerpt_val])]
    if split_target is None:
        return d_train, d_val
    else:
        return d_train.drop('label', axis=1), d_train.label, d_val.drop('label', axis=1), d_val.label
    

In [425]:
X = trans_feat

In [432]:
x1, y1, x2, y2 = train_val_ts_split(X , th=0.7, split_target=1)

In [437]:
X1, X2 = train_val_ts_split(X , th=0.7, split_target=None)

In [ ]:
# внутри датасетов данные не сортированы по времени, но находятся в рамках диапазано.